In [43]:
import pandas as pd
import numpy as np

vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[['country', 'date', 'total_vaccinations', 'people_vaccinated', 'people_vaccinated_per_hundred','people_fully_vaccinated', 'people_fully_vaccinated_per_hundred','daily_vaccinations', 'vaccines']]
#vaccinations_data.head()

# country — наименование страны;
# date — дата наблюдения;
# total_vaccinations — общее число введённых вакцин в стране на указанный день;
# people_vaccinated — общее число привитых первым компонентом в стране на указанный день;
# people_vaccinated_per_hundred — процент привитых первым компонентом в стране на указанный день (рассчитывается как );
# people_fully_vaccinated — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);
# people_fully_vaccinated_per_hundred — процент привитых вторым компонентом в стране на указанный день (рассчитывается как );
# daily_vaccination — ежедневная вакцинация (число вакцинированных в указанный день);
# vaccines — комбинации вакцин, используемые в стране.

covid_data = pd.read_csv('data/covid_data.csv')

# date — дата наблюдения;
# province/state — наименование провинции/штата;
# country — наименование страны;
# confirmed — общее число зафиксированных случаев на указанный день;
# deaths — общее число зафиксированных смертей на указанный день;
# recovered — общее число выздоровлений на указанный день.


#избавимся от штатов, те объединим по стране и дате, столбцы заразились, умели, вылечились просуммируем
covid_data = covid_data.groupby(['date', 'country'], as_index=False)[['confirmed', 'deaths', 'recovered']].sum()
#таблица на 86785 строк

#преобразуем даты
covid_data['date'] = pd.to_datetime(covid_data['date'])
#   date	    country	    confirmed	deaths	recovered
# 0	2021-01-01	Afghanistan	51526.0	    2191.0	41727.0

#создадим новый столбец, которые сейчас именно болеют
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']
covid_data.head()
#   date	    country	    confirmed	deaths	recovered	active
# 0	2021-01-01	Afghanistan	51526.0	    2191.0	41727.0	    7608.0

#Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. 
# Для этого отсортируем данные по названиям стран, а затем по датам. 
# После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода diff():

covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()
#       date	    country	    confirmed	deaths	recovered	active	daily_confirmed	daily_deaths	daily_recovered
# 13426	2020-03-04	Afghanistan	4.0	        0.0	    0.0	        4.0	    2.0	            0.0	            0.0

#в первой таблице преобразуем только даты
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

# 1 Задание За какой период представлены данные в таблице covid_data?
covid_data = covid_data.sort_values(by=['date'])
#display(covid_data.iloc[0]['date'])
#display(covid_data.iloc[86784]['date'])

# 2 Задание За какой период представляены данные в таблице vaccinations_data
vaccinations_data = vaccinations_data.sort_values(by=['date'])
# таблица на 42795 строк кода
#display(vaccinations_data.iloc[0]['date'])
#display(vaccinations_data.iloc[42794]['date'])

# 3 Задание С помощью метода merge() объедините таблицы covid_data и vaccinations_data по столбцам date и country.
#Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, вычисленный в задании 3.1. 
# То есть в результирующую таблицу должны попасть все записи из таблицы covid_data и из её пересечения с vaccinations_data, но не более. 
# Результат объединения занесите в переменную covid_df.

covid_df = covid_data.merge(vaccinations_data, on = ['date', 'country'], how='left')
# таблица на 86785 строк и 16 признаков

# 4 Задание В получившейся в задании 3.3 таблице covid_df создайте признаки death_rate — 
# общий процент смертей среди зафиксированных случаев (летальность) и recover_rate — 
# общий процент случаев выздоровления. Данные характеристики рассчитайте как отношение числа смертей (deaths) и числа выздоровлений (recovered) 
# к числу зафиксированных случаев (confirmed) и умножьте результаты на 100%.

# death_rate = deaths/confirmed * 100
# recover_rate = recovered/confirmed * 100
covid_df['death_rate'] = covid_df['deaths']/covid_df['confirmed'] * 100
covid_df['recover_rate'] = covid_df['recovered']/covid_df['confirmed'] * 100

# 5 Задание Какова максимальная летальность в США (United States) за весь период?
death_rate_USA = round(covid_df[covid_df['country']=='United States']['death_rate'].max(), 2)
print(death_rate_USA)

# 6 Задание Чему равен средний процент выздоровевших в России (Russia)? Ответ округлите до второго знака после запятой.
confirmed_rate_Russian = round(covid_df[covid_df['country']=='Russia']['recover_rate'].mean(), 2)
print(confirmed_rate_Russian)
#сохраним нашу табличку
covid_df.to_csv('Data/covid_df.csv')




7.19
67.06
